# Démonstration Interactive : Cassage de PRNG

**Contexte :** Cette démonstration illustre la vulnérabilité des générateurs pseudo-aléatoires classiques (LCG et Mersenne Twister) face à une attaque par prédiction.

**Objectif :**
1. Retrouver les paramètres secrets d'un LCG.
2. Cloner l'état interne du générateur `random` de Python.
3. Prédire le futur.

---

## 1. Attaque sur le LCG (Linear Congruential Generator)

**Scénario :** Nous sommes face à une "Boîte Noire" qui génère des nombres. Nous ne connaissons ni sa graine (seed), ni ses paramètres $a$ et $c$. Nous connaissons seulement le module $m$ (souvent standard, ex: $2^{31}$).

In [ ]:
# --- MISE EN PLACE DE LA VICTIME ---
class VulnerableLCG:
    def __init__(self, seed, a, c, m):
        self.state = seed
        self.a = a
        self.c = c
        self.m = m

    def next(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state

# Paramètres secrets (l'attaquant ne les connait pas)
SECRET_A = 1103515245
SECRET_C = 12345
SECRET_M = 2**31
SECRET_SEED = 424242

# Création de l'instance cible
target_lcg = VulnerableLCG(SECRET_SEED, SECRET_A, SECRET_C, SECRET_M)
print("Cible LCG active et paramétrée.")

### Phase 1 : Espionnage
L'attaquant écoute le réseau et capture 3 valeurs consécutives.

In [ ]:
# Capture des valeurs
x1 = target_lcg.next()
x2 = target_lcg.next()
x3 = target_lcg.next()

print(f"Valeurs interceptées :")
print(f"X1 = {x1}")
print(f"X2 = {x2}")
print(f"X3 = {x3}")

### Phase 2 : Attaque Algébrique
Nous résolvons le système d'équations :
$$X_2 = aX_1 + c \pmod m$$
$$X_3 = aX_2 + c \pmod m$$

En soustrayant les deux équations, $c$ disparait. On trouve $a$ en calculant l'inverse modulaire.

In [ ]:
def crack_lcg(x1, x2, x3, m):
    # Calcul de l'inverse modulaire de (x2 - x1)
    try:
        inv = pow(x2 - x1, -1, m)
    except ValueError:
        return None, None # Pas inversible (rare)
    
    # Calcul de a
    a_found = ((x3 - x2) * inv) % m
    
    # Calcul de c
    c_found = (x2 - a_found * x1) % m
    
    return a_found, c_found

# Exécution de l'attaque
a_cracked, c_cracked = crack_lcg(x1, x2, x3, SECRET_M)

print(f"Paramètres trouvés :")
print(f"a = {a_cracked}  (Réel: {SECRET_A})")
print(f"c = {c_cracked}      (Réel: {SECRET_C})")

if a_cracked == SECRET_A and c_cracked == SECRET_C:
    print("\n>> SUCCÈS : Le générateur est cassé ! <<")
else:
    print("\n>> ÉCHEC <<")

---

## 2. Attaque sur Mersenne Twister (Python `random`)

**Scénario :** Un site web utilise `random.getrandbits(32)` pour générer des tokens de session. Nous voulons prédire les prochains tokens.

Le Mersenne Twister ne chiffre pas son état, il le mélange via une fonction bijective appelée **Tempering**. Si on inverse ce mélange (**Untempering**), on retrouve l'état interne.

In [ ]:
import random

# --- FONCTIONS D'ATTAQUE (UNTEMPERING) ---
def unshiftRight(x, shift):
    res = x
    for i in range(32):
        res = x ^ res >> shift
    return res

def unshiftLeft(x, shift, mask):
    res = x
    for i in range(32):
        res = x ^ (res << shift & mask)
    return res

def untemper(v):
    """Inverse les opérations de mélange du Mersenne Twister"""
    v = unshiftRight(v, 18)
    v = unshiftLeft(v, 15, 0xefc60000)
    v = unshiftLeft(v, 7, 0x9d2c5680)
    v = unshiftRight(v, 11)
    return v

print("Outils d'attaque chargés.")

### Phase 1 : Collecte de données
Nous avons besoin de 624 nombres de 32 bits pour reconstruire l'état complet du générateur.

In [ ]:
# La Cible : Le module random officiel de Python
# On l'initialise avec une graine inconnue (ex: le temps actuel)
random.seed()

print("Écoute du générateur en cours...", end="")
captured_values = []
for _ in range(624):
    captured_values.append(random.getrandbits(32))
print(" Fait. (624 valeurs capturées)")

### Phase 2 : Clonage de l'état (The Money Shot)
Nous allons reconstruire l'état interne et injecter cet état dans NOTRE propre générateur.

In [ ]:
# 1. Reconstitution de l'état interne (Untempering)
reconstructed_state = [untemper(x) for x in captured_values]

# 2. Injection dans un nouveau générateur (Clone)
# En Python, on peut setter l'état manuellement
clone = random.Random()
state_tuple = (3, tuple(reconstructed_state + [624]), None)
clone.setstate(state_tuple)

print("État interne reconstruit et injecté dans le clone.")

### Phase 3 : Prédiction vs Réalité
Comparons maintenant ce que le **vrai** serveur génère et ce que **notre clone** a prédit.

In [ ]:
print(f"{'IDX':<5} | {'RÉEL (Serveur)':<20} | {'PRÉDICTION (Moi)':<20} | {'VERDICT'}")
print("-"*65)

for i in range(10):
    real_val = random.getrandbits(32)
    pred_val = clone.getrandbits(32)
    
    status = "MATCH" if real_val == pred_val else "FAIL"
    print(f"{i:<5} | {real_val:<20} | {pred_val:<20} | {status}")

## Conclusion
Nous avons démontré qu'il est possible de **cloner totalement** le générateur aléatoire par défaut de Python. 

**Recommandation :** Utilisez toujours `secrets` pour la cryptographie.

In [ ]:
import secrets
print(f"Token sécurisé : {secrets.token_hex(16)}")